# Azure AI Agent service - Bing integration

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [5]:
import datetime
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import BingGroundingTool, ListSortOrder, MessageRole
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

In [6]:
load_dotenv(".env")

False

## Grounding Bing Search

In [7]:
project_client = AIProjectClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

### Create connection to bing grounding service

In [8]:
# Shoud be: /subscriptions/<subscription_id>/resourceGroups/<resource_group_name>/providers/Microsoft.CognitiveServices/accounts/<ai_service_name>/projects/<project_name>/con#nections/<connection_name>
bing_connection = project_client.connections.get(name=os.environ["BING_CONNECTION_NAME"])
BING_CONN_ID = bing_connection.id
model = os.getenv("MODEL_DEPLOYMENT_NAME")

In [ ]:
bing = BingGroundingTool(connection_id=BING_CONN_ID)
bing


### Create agent with Bing Grounding tool

In [ ]:
# Create the agent with the Bing grounding tool
agents_client = project_client.agents
agent = agents_client.create_agent(
    model=model,
    temperature=0.7,
    name="Bing-grounding-agent-demo",
    instructions="You are an AI Agent that can do some web search using Bing",
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"},
    description="Agent with Grounding Bing Search",
    )

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_XbYA2x5RMy8GXc4N36m8KDlx


### Example 1

In [14]:
prompt = "What is the current price of the MICROSOFT share in USD?"

In [15]:
# Step 1: Create a thread
thread = agents_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = agents_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_N2eEI4QbQRanFsvYtFE6MeFe
💬 [2] Created message | ID: msg_ECUMpHoMRNhCVhRXgWHH3gal
🏃 [3] Created run     | ID: run_i4pxiMqkwrJMBEjo13CnJo8p
⏳ [4] Polling run status...
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.COMPLETED

🔍 Step ID details: step_GwOvKl5hA3F2svOKdhUnEh6V, Type: RunStepType.MESSAGE_CREATION
Details step: {'id': 'step_GwOvKl5hA3F2svOKdhUnEh6V', 'object': 'thread.run.step', 'created_at': 1756103342, 'run_id': 'run_i4pxiMqkwrJMBEjo13CnJo8p', 'assistant_id': 'asst_XbYA2x5RMy8GXc4N36m8KDlx', 'thread_id': 'thread_N2eEI4QbQRanFsvYtFE6MeFe', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1756103342, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_3pjODIqtxcmkcizMJjjrReK2'}}, 'usage': {'prompt_tokens': 2053, 'completion_tokens': 47, 'total_tokens': 2100, 'prompt_token_de

## Clean Up: Delete Agent

In [16]:
agents_client.delete_agent(agent.id)
print(f"Deleted agent, agent ID: {agent.id}")

Deleted agent, agent ID: asst_XbYA2x5RMy8GXc4N36m8KDlx
